In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('weatherAUS.csv')

Pre - Processing of the dataset is needed to be done

In [3]:
df.replace('NA', pd.NA, inplace=True)
df.replace('nan', pd.NA, inplace=True)

In [4]:
# Initial view of the dataset
print(df.head)

<bound method NDFrame.head of               Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  \
0       2008-12-01   Albury     13.4     22.9       0.6          NaN   
1       2008-12-02   Albury      7.4     25.1       0.0          NaN   
2       2008-12-03   Albury     12.9     25.7       0.0          NaN   
3       2008-12-04   Albury      9.2     28.0       0.0          NaN   
4       2008-12-05   Albury     17.5     32.3       1.0          NaN   
...            ...      ...      ...      ...       ...          ...   
145455  2017-06-21    Uluru      2.8     23.4       0.0          NaN   
145456  2017-06-22    Uluru      3.6     25.3       0.0          NaN   
145457  2017-06-23    Uluru      5.4     26.9       0.0          NaN   
145458  2017-06-24    Uluru      7.8     27.0       0.0          NaN   
145459  2017-06-25    Uluru     14.9      NaN       0.0          NaN   

        Sunshine WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  \
0            NaN           W  

In [5]:
from sklearn.preprocessing import LabelEncoder

# List of columns to drop from the DataFrame
columns_to_drop = ['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm', 'Date']

# Drop specified columns from the DataFrame
df = df.drop(columns=columns_to_drop)

# Drop rows where any remaining columns have null values
df = df.dropna()
print(df.head)

# List of columns to encode using LabelEncoder
columns_to_encode = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']

# Initialize LabelEncoder and encode each column in columns_to_encode list
for column in columns_to_encode:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])

<bound method NDFrame.head of        Location  MinTemp  MaxTemp  Rainfall WindGustDir  WindGustSpeed  \
0        Albury     13.4     22.9       0.6           W           44.0   
1        Albury      7.4     25.1       0.0         WNW           44.0   
2        Albury     12.9     25.7       0.0         WSW           46.0   
3        Albury      9.2     28.0       0.0          NE           24.0   
4        Albury     17.5     32.3       1.0           W           41.0   
...         ...      ...      ...       ...         ...            ...   
145454    Uluru      3.5     21.8       0.0           E           31.0   
145455    Uluru      2.8     23.4       0.0           E           31.0   
145456    Uluru      3.6     25.3       0.0         NNW           22.0   
145457    Uluru      5.4     26.9       0.0           N           37.0   
145458    Uluru      7.8     27.0       0.0          SE           28.0   

       WindDir9am WindDir3pm  WindSpeed9am  WindSpeed3pm  Humidity9am  \
0       

In [6]:
# Dataset after dropping columns and removing rows with NaN cells
print(df.head)

<bound method NDFrame.head of         Location  MinTemp  MaxTemp  Rainfall  WindGustDir  WindGustSpeed  \
0              1     13.4     22.9       0.6           13           44.0   
1              1      7.4     25.1       0.0           14           44.0   
2              1     12.9     25.7       0.0           15           46.0   
3              1      9.2     28.0       0.0            4           24.0   
4              1     17.5     32.3       1.0           13           41.0   
...          ...      ...      ...       ...          ...            ...   
145454        36      3.5     21.8       0.0            0           31.0   
145455        36      2.8     23.4       0.0            0           31.0   
145456        36      3.6     25.3       0.0            6           22.0   
145457        36      5.4     26.9       0.0            3           37.0   
145458        36      7.8     27.0       0.0            9           28.0   

        WindDir9am  WindDir3pm  WindSpeed9am  WindSpeed3p

In [7]:
# Select numerical features for outlier detection
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Calculate quartiles and IQR
Q1 = df[numerical_features].quantile(0.25)
Q3 = df[numerical_features].quantile(0.75)
IQR = Q3 - Q1

# Detect outliers using IQR
outliers = ((df[numerical_features] < (Q1 - 1.5 * IQR)) | (df[numerical_features] > (Q3 + 1.5 * IQR))).any(axis=1)

# Remove outliers
df_cleaned = df[~outliers]

In [8]:
# Write the pre-processed DataFrame to a CSV file
df.to_csv('modified_dataset.csv', index=False)

Pre-Processing of the dataset is done.

In [9]:
# Load the pre-processed dataset
df = pd.read_csv('modified_dataset.csv')

# Separate features (X) and target variable (y)
X = df.drop(columns=['RainTomorrow'])
y = df['RainTomorrow']

Now, we do for `Linear models: Logistic Regression, Naïve Bayes`

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [11]:
# Before tuning the hyperparameters

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training set, transform the testing set
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize Logistic Regression model
logistic_model = LogisticRegression(max_iter=2000, solver='liblinear')

# Fit Logistic Regression model
logistic_model.fit(X_train_scaled, y_train)

# Evaluate Logistic Regression model
logistic_train_accuracy = logistic_model.score(X_train_scaled, y_train)
logistic_test_accuracy = logistic_model.score(X_test_scaled, y_test)

# Initialize Naïve Bayes model
naive_bayes_model = GaussianNB()

# Fit Naïve Bayes model
naive_bayes_model.fit(X_train_scaled, y_train)

# Evaluate Naïve Bayes model
naive_bayes_train_accuracy = naive_bayes_model.score(X_train_scaled, y_train)
naive_bayes_test_accuracy = naive_bayes_model.score(X_test_scaled, y_test)

# Print results
print("Before tuning hyperparameters:")
print("Linear Model: Logistic Regression")
print("Training Accuracy:", logistic_train_accuracy)
print("Testing Accuracy:", logistic_test_accuracy)

print("\nNaïve Bayes Classifier")
print("Training Accuracy:", naive_bayes_train_accuracy)
print("Testing Accuracy:", naive_bayes_test_accuracy)


# After tuning the hyperparameters

# Initialize Logistic Regression model with tuned hyperparameters
logistic_model_tuned = LogisticRegression(max_iter=1000, solver='lbfgs', C=1.0)

# Fit Logistic Regression model with tuned hyperparameters
logistic_model_tuned.fit(X_train_scaled, y_train)

# Evaluate Logistic Regression model with tuned hyperparameters
logistic_train_accuracy_tuned = logistic_model_tuned.score(X_train_scaled, y_train)
logistic_test_accuracy_tuned = logistic_model_tuned.score(X_test_scaled, y_test)

# Print results after tuning
print("\nAfter tuning hyperparameters:")
print("Linear Model: Logistic Regression (Tuned)")
print("Training Accuracy:", logistic_train_accuracy_tuned)
print("Testing Accuracy:", logistic_test_accuracy_tuned)

# Gaussian Naïve Bayes typically doesn't require tuning hyperparameters,

Before tuning hyperparameters:
Linear Model: Logistic Regression
Training Accuracy: 0.8496347133052912
Testing Accuracy: 0.8475536860748284

Naïve Bayes Classifier
Training Accuracy: 0.8119216294000443
Testing Accuracy: 0.8093867611246403

After tuning hyperparameters:
Linear Model: Logistic Regression (Tuned)
Training Accuracy: 0.849623644011512
Testing Accuracy: 0.8475536860748284


Now, we do for `Non Linear models: Decision tress, Neural Network`

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

In [13]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training set, transform the testing set
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize Decision Tree model
decision_tree_model = DecisionTreeClassifier(random_state=0)

# Fit Decision Tree model
decision_tree_model.fit(X_train_scaled, y_train)

# Evaluate Decision Tree model before tuning
decision_tree_train_accuracy_before = decision_tree_model.score(X_train_scaled, y_train)
decision_tree_test_accuracy_before = decision_tree_model.score(X_test_scaled, y_test)

# Define a smaller search space for Decision Tree hyperparameters
param_grid_decision_tree = {'max_depth': [None, 10, 20]}

# Use RandomizedSearchCV for hyperparameter tuning
random_search_decision_tree = RandomizedSearchCV(DecisionTreeClassifier(random_state=0),
                                                param_distributions=param_grid_decision_tree,
                                                n_iter=3, cv=3, random_state=0)
random_search_decision_tree.fit(X_train_scaled, y_train)
decision_tree_model_tuned = random_search_decision_tree.best_estimator_

# Evaluate Decision Tree model after tuning
decision_tree_train_accuracy_after = decision_tree_model_tuned.score(X_train_scaled, y_train)
decision_tree_test_accuracy_after = decision_tree_model_tuned.score(X_test_scaled, y_test)

# Initialize Neural Network model
neural_network_model = MLPClassifier(random_state=0)

# Fit Neural Network model
neural_network_model.fit(X_train_scaled, y_train)

# Evaluate Neural Network model before tuning
neural_network_train_accuracy_before = neural_network_model.score(X_train_scaled, y_train)
neural_network_test_accuracy_before = neural_network_model.score(X_test_scaled, y_test)

# Define a smaller search space for Neural Network hyperparameters
param_grid_neural_network = {'hidden_layer_sizes': [(50,), (100,)]}

# Use RandomizedSearchCV for hyperparameter tuning
random_search_neural_network = RandomizedSearchCV(MLPClassifier(random_state=0),
                                                  param_distributions=param_grid_neural_network,
                                                  n_iter=2, cv=3, random_state=0)
random_search_neural_network.fit(X_train_scaled, y_train)
neural_network_model_tuned = random_search_neural_network.best_estimator_

# Evaluate Neural Network model after tuning
neural_network_train_accuracy_after = neural_network_model_tuned.score(X_train_scaled, y_train)
neural_network_test_accuracy_after = neural_network_model_tuned.score(X_test_scaled, y_test)

# Print results
print("Decision Tree Classifier:")
print("Before tuning hyperparameters:")
print("Training Accuracy:", decision_tree_train_accuracy_before)
print("Testing Accuracy:", decision_tree_test_accuracy_before)
print("\nAfter tuning hyperparameters:")
print("Training Accuracy:", decision_tree_train_accuracy_after)
print("Testing Accuracy:", decision_tree_test_accuracy_after)

print("\nNeural Network Classifier:")
print("Before tuning hyperparameters:")
print("Training Accuracy:", neural_network_train_accuracy_before)
print("Testing Accuracy:", neural_network_test_accuracy_before)
print("\nAfter tuning hyperparameters:")
print("Training Accuracy:", neural_network_train_accuracy_after)
print("Testing Accuracy:", neural_network_test_accuracy_after)

/opt/homebrew/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Decision Tree Classifier:
Before tuning hyperparameters:
Training Accuracy: 1.0
Testing Accuracy: 0.7880008855435023

After tuning hyperparameters:
Training Accuracy: 0.8707106486606154
Testing Accuracy: 0.8437458490148328

Neural Network Classifier:
Before tuning hyperparameters:
Training Accuracy: 0.8678326322780606
Testing Accuracy: 0.8574717733008634

After tuning hyperparameters:
Training Accuracy: 0.8636373699357981
Testing Accuracy: 0.8580916537524906


Now, we do for `Hybrid models: SVM, Bayesian Network`

In [14]:
from sklearn.svm import SVC
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, BayesianEstimator
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [9]:
# Load the dataset
data = pd.read_csv("modified_dataset.csv")

# Separate features (X) and target variable (y)
X = data.drop(columns=["Location", "RainTomorrow"])
y = data["RainTomorrow"]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training set, transform the testing set
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [15]:
# Initialize SVM model
svm_model = SVC(random_state=0)

# Fit SVM model
svm_model.fit(X_train_scaled, y_train)

# Evaluate SVM model before tuning
svm_train_accuracy_before = svm_model.score(X_train_scaled, y_train)
svm_test_accuracy_before = svm_model.score(X_test_scaled, y_test)

# Print results before tuning
print("Support Vector Machine (SVM) Classifier:")
print("Before tuning hyperparameters:")
print("Training Accuracy:", svm_train_accuracy_before)
print("Testing Accuracy:", svm_test_accuracy_before)

# Define a smaller search space for SVM hyperparameters
param_grid_svm = {'C': [1, 10], 'gamma': [0.1, 'scale']}

# Use GridSearchCV for hyperparameter tuning
grid_search_svm = GridSearchCV(SVC(random_state=0), param_grid=param_grid_svm, cv=2, n_jobs=-1)
grid_search_svm.fit(X_train_scaled, y_train)
svm_model_tuned = grid_search_svm.best_estimator_

# Evaluate SVM model after tuning
svm_train_accuracy_after = svm_model_tuned.score(X_train_scaled, y_train)
svm_test_accuracy_after = svm_model_tuned.score(X_test_scaled, y_test)

# Print results after tuning
print("\nAfter tuning hyperparameters:")
print("Training Accuracy:", svm_train_accuracy_after)
print("Testing Accuracy:", svm_test_accuracy_after)

Support Vector Machine (SVM) Classifier:
Before tuning hyperparameters:
Training Accuracy: 0.8622315696258579
Testing Accuracy: 0.8572946646003985

After tuning hyperparameters:
Training Accuracy: 0.8763117113128183
Testing Accuracy: 0.8599955722824884


Bayesian networks don't typically have hyperparameters in the same sense as other machine learning models like SVM or neural networks.

Instead of hyperparameter tuning, the focus in Bayesian networks is often on selecting the best structure and estimating its parameters given the data.

In [42]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split

In [43]:
# Read data from CSV file
data = pd.read_csv("modified_dataset.csv")

# Specify the desired sample size
sample_size = 1000  # Adjust this value according to your requirements

# Randomly sample from the dataset
smaller_data = data.sample(n=sample_size, random_state=42)

# Save the smaller dataset to a new CSV file
smaller_data.to_csv("smaller_dataset.csv", index=False)


In [ ]:
# Read data from CSV file
data = pd.read_csv("smaller_dataset.csv")
selected_columns = ['Humidity3pm', 'RainToday', 'MaxTemp', 'Rainfall', 'RainTomorrow']
data_subset = data[selected_columns]

# Define features and target variable
X = data_subset[['Humidity3pm', 'RainToday', 'MaxTemp', 'Rainfall']]
y = data_subset['RainTomorrow']

selected_columns = ['Humidity3pm', 'RainToday', 'MaxTemp', 'Rainfall']
data_subset = data_subset[selected_columns]

# Print the variable names
variable_names = X.columns.tolist()
print("Variable names used and passed to the model:", variable_names)
print("Columns in data: " + str(list(data_subset.columns)))

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the structure of the simplified Bayesian Network
model = BayesianNetwork([
    ('Humidity3pm', 'RainTomorrow'),
    ('RainToday', 'RainTomorrow'),
    ('MaxTemp', 'RainTomorrow'),
    ('Rainfall', 'RainTomorrow')
])

# Estimate the parameters of the model using BayesianEstimator
model.fit(X_train, estimator=BayesianEstimator, n_jobs=-1)

# Predict on test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = (y_pred['RainTomorrow'] == y_test).mean()
print("Accuracy:", accuracy)
